# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Diözesen übertragen

Lies Daten aus `Kloester und Stifte_neu_2020.accdb`.

- [Diözesen](#Diözesen)
  - [Einträge in `item`](#Einträge-in-item)
  - [Externe URLs eintragen](#Externe-URLs-eintragen)
  - [Alternative Namen eintragen](#Alternative-Namen-eintragen)
  - [Bischofssitze zuordnen](#Bischofssitze-zuordnen)
  - [Literaturverweise](#Literaturverweise)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise

In [6]:
using WiagDataSetup

In [7]:
using MySQL, CSV, DataFrames

In [9]:
includet("src/Bistum.jl")

In [10]:
Wds = WiagDataSetup

WiagDataSetup

In [11]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

Trage die jeweils aktuelle Datenquelle ein!

In [12]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bistümer-2022-11-10"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bistümer-2022-11-10"

In [17]:
item_type_id = 1

1

In [18]:
check_globals()

┌ Info: WiagDataSetup
└ @ Main C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\Bistum.jl:13
┌ Info: 
│   item_type_id = 1
└ @ Main C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\Bistum.jl:14


## Diözesen

In [15]:
csv_file = joinpath(data_path, "gs_dioceses.csv")
df_d = read_diocese(csv_file, :id_diocese);

┌ Info: Zeilen, Spalten: 
│   size(df_d) = (119, 16)
└ @ Main C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\Bistum.jl:36


## Bischofssitze
*2022-11-10* Bischofssitze neu zuordnen. Die bisherige Zuordnung nutzte die Namen der Orte, die freilich nicht eindeutig sind. Bei manchen Bischofssitzen ist die falsche GeoNames ID eingetragen (feature code "A" statt "P"). Das ist in der editierten Version korrigiert.

In [88]:
csv_file = joinpath(data_path, "gs_places_edit.csv")
df_p = read_place(csv_file);

┌ Info: Zeilen, Spalten: 
│   size(df_p) = (46921, 16)
└ @ Main C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\Bistum.jl:53


Ergänze `df_d` um `geonames_id`.

In [89]:
df_d_p = leftjoin(
    df_d, 
    select(df_p, [:id_places => :id_places, :place_name => :bs_name, :geonames_id => :geonames_id]), 
    on = :bishopric_seat => :id_places, 
    matchmissing = :notequal);

Lies Orte aus WIAG

In [90]:
table = "place";
sql = "select id, name, geonames_id from $(table)";
df_p_wiag = Wds.sql_df(sql);

Ergänze `df_d_p` um `place.id` aus WIAG

In [91]:
df_d_p = leftjoin(
    df_d_p,
    select(df_p_wiag, [:id => :bishopric_seat_id, :name => :wiag_bs_name, :geonames_id => :geonames_id]),
    on = :geonames_id,
    matchmissing = :notequal);

In [92]:
df_d_p[21:25, [:id_diocese, :diocese, :bs_name, :wiag_bs_name, :geonames_id, :bishopric_seat_id]]

Row,id_diocese,diocese,bs_name,wiag_bs_name,geonames_id,bishopric_seat_id
,String,String,String?,String?,Int64?,Int32?
1,14,Lübeck,Lübeck,Lübeck,2875601,386543
2,15,Magdeburg,Magdeburg,Magdeburg,2874545,387084
3,16,Mainz,Mainz,Mainz,2874225,387241
4,13,Lebus,Lebus,Lebus,2879795,388462
5,11,Köln,Köln,Köln,2886242,391607


Lies Bistümer aus WIAG

In [93]:
table = "diocese";
sql = "select id, name from $(table)";
df_d_wiag = Wds.sql_df(sql);

Ergänze `df_d_wiag` um einen neuen Wert für `bishopric_seat`

In [94]:
df_d_wiag = leftjoin(
    df_d_wiag,
    select(df_d_p, [:diocese, :bs_name, :wiag_bs_name, :geonames_id, :bishopric_seat_id]),
    on = :name => :diocese,
);

Keine Fehlermeldung: Alle Diözesen konnten zugeordnet werden

In [95]:
size(df_d_wiag)

(79, 6)

In [103]:
filter(:geonames_id => ismissing, df_d_wiag)

Row,id,name,bs_name,wiag_bs_name,geonames_id,bishopric_seat_id
,Int32,String?,String?,String?,Int64?,Int32?
1,55,Nordische Missionen,missing,missing,missing,missing


In [105]:
dropmissing!(df_d_wiag, :bs_name);

In [106]:
table = "diocese_update";
sql = "DROP TABLE IF EXISTS $(table)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DROP TABLE IF EXISTS diocese_update", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [107]:
table = "diocese_update"
sql = "CREATE TEMPORARY TABLE $(table) " *
"(id INT, bishopric_seat_id INT) " *
"DEFAULT CHARSET=utf8";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "CREATE TEMPORARY TABLE diocese_update (id INT, bishopric_seat_id INT) DEFAULT CHARSET=utf8", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [108]:
table = "diocese_update"
Wds.filltable!(table, select(df_d_wiag, [:id, :bishopric_seat_id]))

┌ Info: Rows inserted: 78
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


78

In [109]:
sql = "UPDATE diocese, (SELECT id, bishopric_seat_id FROM diocese_update) as d_update " *
"SET diocese.bishopric_seat_id = d_update.bishopric_seat_id " *
"WHERE diocese.id = d_update.id";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "UPDATE diocese, (SELECT id, bishopric_seat_id FROM diocese_update) as d_update SET diocese.bishopric_seat_id = d_update.bishopric_seat_id WHERE diocese.id = d_update.id", 0, -1, 77, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

### Einträge in `item`

entsprechende Einträge in Tabelle `item` einfügen

In [10]:
df_item = select(df_d, :id_diocese => :id_in_source);

In [11]:
user_wiag_id = 40;

In [12]:
using Dates

In [13]:
Dates.now()

2022-11-10T09:20:14.997

In [ ]:
insertcols!(
    df_item, 
    :item_type_id => 1,
    :edit_status => "online",
    :is_online => true,
    :created_by => user_wiag_id,
    :changed_by => user_wiag_id,
    
)

In [37]:
df_item[!, :item_type_id] .= 1;

In [38]:
df_item[!, :edit_status] .= "online";

In [ ]:
df_item[!, :is_online] .= true;

In [39]:
df_item[!, :created_by] .= 1;

In [40]:
df_item[!, :date_created] .= "2021-09-30 10:20";

In [41]:
df_item[!, :changed_by] .= 1;

In [42]:
df_item[!, :date_changed] .= "2021-09-30 10:20";

In [43]:
function make_id_public(id::Real, num_id_length::Int, id_public_key::AbstractString)
    return "WIAG-" * id_public_key * "-" * lpad(id, num_id_length, '0') * "-001"
end

make_id_public (generic function with 1 method)

In [44]:
make_id_public(14, 3, "Inst-DIOCGatz")

"WIAG-Inst-DIOCGatz-014-001"

Normdatum/öffentliche ID erzeugen

Verwende die ursprüngliche ID. Damit entspricht die ID der bisher veröffentlichten ID (2021-09-30)

In [45]:
df_item[!, :id_public] .= ((id) -> make_id_public(id, 3, "Inst-DIOCGatz")).(df_item[!, :id_in_source]);

`item.is_deleted` ist vorbelegt mit `false`

`item.lang` ist vorbelegt mit "de"

In [46]:
names(df_item)

8-element Vector{String}:
 "source_id"
 "item_type_id"
 "edit_status"
 "created_by"
 "date_created"
 "changed_by"
 "date_changed"
 "id_public"

In [47]:
df_item[1:5, [:id_in_source, :item_type_id, :id_public]]

,source_id,item_type_id,id_public
,Int32,Int64,String
1,1,1,WIAG-Inst-DIOCGatz-001-001
2,2,1,WIAG-Inst-DIOCGatz-002-001
3,3,1,WIAG-Inst-DIOCGatz-003-001
4,4,1,WIAG-Inst-DIOCGatz-004-001
5,5,1,WIAG-Inst-DIOCGatz-005-001


In [80]:
Wds.filltable!("item", df_item)

┌ Info: Rows inserted: 79
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


79

### Daten eintragen

IDs auslesen

In [15]:
sql = "SELECT id, id_in_source FROM item where item_type_id = 1"
df_idx = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

Spalte für die Zusammenführung befüllen (Typ: String!)

In [16]:
df_dioc[!, :id_in_source] .= string.(df_dioc[!, :id_diocese]);

In [17]:
df_dioc_db = innerjoin(df_idx, df_dioc, on = :id_in_source);

In [84]:
df_dioc_db[5:10, [:id, :name, :diocese_status]]

,id,name,diocese_status
,Int32,String?,String?
1,5,Chiemsee,Bistum
2,6,Eichstätt,Bistum
3,7,Freising,Bistum
4,8,Halberstadt,Bistum
5,9,Havelberg,Bistum
6,10,Hildesheim,Bistum


In [85]:
select_cols = [
    :id,
    :diocese => :name,
    :note_diocese => :comment,
    :ecclesiastical_province,
    :diocese_status,
    :shapefile,
    :bishopric_seat,
    :note_bishopric_seat => :comment_bishopric_seat,
    :date_of_founding,
    :date_of_dissolution,
    :comment_authority_file,
    :altes_reich => :is_altes_reich,
    :diocese_gs => :is_diocese_gs
];

In [94]:
Wds.filltable!("diocese", select(df_dioc_db, Not([:gatz_pages, :id_in_source, :id_diocese])); clear_table=true)

┌ Info: Rows inserted: 79
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


79

### Externe URLs eintragen

In [156]:
Wds.setDBWIAG(user = "georg", db = "wiag2")

Passwort für User georg
stdin> topf&deckel


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

Quelle: Tabelle `wiag.id_external_urls_diocese`

IDs auslesen

In [113]:
sql = "SELECT id as item_id, source_id FROM ITEM where item_type_id = 1"
df_idx = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [128]:
size(df_idx)

(79, 2)

Daten auslesen

In [120]:
sql = "SELECT diocese_id, " * 
    "comment, " * 
    "url_type_id as authority_id, " * 
    "url_value as value " *
    "FROM wiag.id_external_urls_diocese"
df_ext_url = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [121]:
size(df_ext_url)

(519, 4)

Spalte für die Zusammenführung befüllen (Typ: String!)

In [122]:
df_ext_url[!, :id_in_source] .= string.(df_ext_url[!, :diocese_id]);

In [150]:
df_ext_url_db = innerjoin(df_idx, df_ext_url, on = :id_in_source);

In [151]:
size(df_ext_url_db)

(450, 6)

In der Quelle gibt es 512 Einträge, aber nur 450 sind einer der 79 Diözesen zuzuordnen.

In [157]:
Wds.filltable!("id_external", select(df_ext_url_db, Not([:id_in_source, :diocese_id])))

┌ Info: Rows inserted: 450
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


450

### Alternative Namen eintragen

Quelle: Tabelle `wiag.alt_label_diocese`

IDs auslesen

In [5]:
using MySQL, DataFrames

In [17]:
sql = "SELECT id as concept_id, id_in_source FROM ITEM where item_type_id = 1"
df_idx = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [176]:
size(df_idx)

(79, 2)

Name als bevorzugte Bezeichnung/Label eintragen

In [27]:
scheme_id = 1
sql = "SELECT id AS concept_id, CONCAT(diocese_status, ' ', name) AS label, $(scheme_id) AS skos_scheme_id, " *
"'de' AS lang, 1 AS display_order, 1 AS is_preferred " *
"FROM diocese"
df_dioc_name = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [28]:
df_dioc_name[1:5, :]

,concept_id,label,skos_scheme_id,lang,display_order,is_preferred
,Int32,String?,Int64,String,Int64,Int64
1,1,Bistum Augsburg,1,de,1,1
2,2,Bistum Bamberg,1,de,1,1
3,3,Bistum Brandenburg,1,de,1,1
4,4,Erzbistum Bremen-Hamburg,1,de,1,1
5,5,Bistum Chiemsee,1,de,1,1


In [30]:
names(df_dioc_name)

6-element Vector{String}:
 "concept_id"
 "label"
 "skos_scheme_id"
 "lang"
 "display_order"
 "is_preferred"

In [31]:
Wds.filltable!("skos_label", df_dioc_name)

┌ Info: Rows inserted: 79
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


79

alternative Bezeichnungen/Label einlesen

In [177]:
scheme_id = 1
sql = "SELECT diocese_id, " *
string(scheme_id) * " as skos_scheme_id, " *
"alt_label_diocese as label, " *
" lang, " * 
"ressource as note, " * 
"sort as display_order " *
"FROM wiag.alt_label_diocese"
df_alt_label = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [170]:
first(df_alt_label, 5)

,diocese_id,skos_scheme_id,label,lang,note
,Int32?,Int64,String?,String?,String?
1,1,1,ecclesia Augustana,la,"Gatz, Die Bistümer des Alten Reiches"
2,2,1,ecclesia Bambergensis,la,"Gatz, Die Bistümer des Alten Reiches"
3,3,1,ecclesia Brandeburgensis,la,"Gatz, Die Bistümer des Alten Reiches"
4,4,1,ecclesia Bremensis,la,"Gatz, Die Bistümer des Alten Reiches"
5,5,1,ecclesia Chiemensis,la,"Gatz, Die Bistümer des Alten Reiches"


Spalte für die Zusammenführung befüllen (Typ: String!)

In [179]:
df_alt_label[!, :id_in_source] .= string.(df_alt_label[!, :diocese_id]);

In [180]:
df_alt_label_db = innerjoin(df_idx, df_alt_label, on = :id_in_source);

In [181]:
df_alt_label_db[17:23, :]

,concept_id,source_id,diocese_id,skos_scheme_id,label,lang
,Int32,String?,Int32?,Int64,String?,String?
1,17,17,17,1,ecclesia Misnensis,la
2,18,18,18,1,ecclesia Merseburgensis,la
3,19,19,19,1,ecclesia Mindensis,la
4,20,20,20,1,ecclesia Monasteriensis,la
5,21,21,21,1,ecclesia Nuemburgensis,la
6,22,22,22,1,ecclesia Osnabrugensis,la
7,23,23,23,1,ecclesia Paderbornensis,la


In [182]:
names(df_alt_label_db)

8-element Vector{String}:
 "concept_id"
 "source_id"
 "diocese_id"
 "skos_scheme_id"
 "label"
 "lang"
 "note"
 "display_order"

In [184]:
Wds.filltable!("skos_label", select(df_alt_label_db, Not([:id_in_source, :diocese_id])))

┌ Info: Rows inserted: 486
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


486

In der Quelle gibt es 521 Einträge, aber nur 486 sind einer der 79 Diözesen zuzuordnen.

### Bischofssitze zuordnen

Die Ortsdaten werden bezogen von GeoNames (siehe Strukturdaten). Die Bischofssitze werden anhand der Koordinaten in der bestehenden Ortstabelle den Orten in der neuen Tabelle `place` zugeordnet.

Package: https://github.com/KristofferC/NearestNeighbors.jl

In [11]:
using NearestNeighbors, MySQL, CSV, DataFrames

In [15]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg
stdin> topf&deckel


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [28]:
sql = "SELECT id, name as new_name, population, longitude, latitude " *
"FROM place"
df_coo = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [29]:
df_coo[10:15, :]

,id,new_name,population,longitude,latitude
,Int32,String?,Int32?,Dec64…?,Dec64…?
1,350343,Zabitz,0,11.71533,51.60745
2,350344,Zabitz,549,11.92419,51.78503
3,350345,Zach,0,11.46667,47.6
4,350346,Zachariae,0,13.08049,53.88519
5,350347,Zachenberg,2173,13.0,48.96667
6,350348,Zachenöd,0,12.8365,48.5852


In [30]:
data_coo = transpose(hcat(df_coo[!, :longitude], df_coo[!, :latitude]));

In [31]:
nn_tree = BallTree(data_coo)

BallTree{StaticArrays.SVector{2, DecFP.Dec64}, 2, DecFP.Dec64, Euclidean}
  Number of points: 391240
  Dimensions: 2
  Metric: Euclidean(0.0)
  Reordered: true

Suche nach einem nächsten Nachbarn testen

In [23]:
df_coo[137, :]

,id,new_name,longitude,latitude
,Int32,String?,Dec64…?,Dec64…?
137,350470,Zauppenberg,11.37942,49.82023


In [25]:
NearestNeighbors.nn(nn_tree, [11.38, 49.815])

(137, 0.005262062333344218)

Diözesen mit den zugeordneten Bischofssitzen samt Koordinaten auslesen.

Quelle: `wiag2.diocese` und `wiag.place`

In [20]:
sql = "SELECT item.id, id_in_source, " * 
    "d1.diocese as name, d1.bishopric_seat as bs_source, " * 
    "p.place_name, p.longitude, p.latitude " *
    "FROM item " * 
    "JOIN wiag.diocese as d1 ON item.id_in_source = d1.id_diocese " *
    "JOIN wiag.place as p ON d1.bishopric_seat = p.id_places";
df_map = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [21]:
size(df_map)

(78, 7)

Für das Bistum 'Nordische Missionen' gibt es keine Angabe zum Bischofssitz.

Koordinaten in Zahlen umwandeln

In [22]:
df_map[!, :num_longitude] .= parse.(Float64, df_map[!, :longitude]);

In [23]:
df_map[!, :num_latitude] .= parse.(Float64, df_map[!, :latitude]);

In [24]:
df_map[!, :new_place_id] .= 0;

In [25]:
names(df_map)

10-element Vector{String}:
 "id"
 "id_in_source"
 "name"
 "bs_source"
 "place_name"
 "longitude"
 "latitude"
 "num_longitude"
 "num_latitude"
 "new_place_id"

Finde für die Ortsangaben zum Bischofssitz einer Diözese den nächstgelegenen Ort unter den Orten aus GeoNames.

In [26]:
rq = df_map[17, :]

,id,id_in_source,name,bs_source,place_name,longitude,latitude
,Int32,String?,String?,Int32,String?,String?,String?
17,17,17,Meißen,40867,Meißen,8.9505,52.2688333333333


In [32]:
name = rq[:name]
df_coo_loop = filter(:new_name => isequal(name), df_coo)

,id,new_name,population,longitude,latitude
,Int32,String?,Int32?,Dec64…?,Dec64…?
1,384390,Meißen,0,8.95489,52.26984


In [55]:
coo = collect(rq[[:longitude, :latitude]])

2-element Vector{String}:
 "8.9505"
 "52.2688333333333"

Suche einen Ort gleichen Namens. Prüfe, ob der Ort in der Nähe liegt. In GeoNames sind manche Orte nur mit ihrem englischen Namen hinterlegt. Wenn es mehrere Treffer gibt, wähle den nächstgelegenen Ort.

Weise dem Feld `bishpric_seat` die ID aus der Tabelle `place` zu.

In [35]:
using LinearAlgebra
function find_dioc_place(row)
    name = row[:name]
    df_coo_loop = filter(:new_name => isequal(name), df_coo)
    n_match = size(df_coo_loop, 1)
    id_place = 0
    coo = collect(row[[:num_longitude, :num_latitude]])
    if n_match == 0
        @warn "Kein Treffer für " * string(row[:id]) * " " * row[:name]
    elseif n_match == 1
        coo_place = collect(df_coo_loop[1, [:longitude, :latitude]])
        coo_delta = coo .- coo_place
        dist = norm(coo_delta)
        if dist > 0.05
            @warn "großer Abstand " * string(dist) * " für " * string(row[:id]) * " " * row[:name]
        end            
        id_place = df_coo_loop[1, :id]
    else
        
        data_coo_loop = transpose(hcat(df_coo_loop[!, :longitude], df_coo_loop[!, :latitude]));
        tree_loop = NearestNeighbors.BallTree(data_coo_loop)
        idx, dist = NearestNeighbors.nn(tree_loop, coo)
        id_place = df_coo_loop[idx, :id]
    end    
    # println(id_place)
    return id_place
end

find_dioc_place (generic function with 1 method)

In [36]:
for (i, row) in enumerate(eachrow(df_map))
    df_map[i, :new_place_id] = find_dioc_place(row)
end

┌ Warning: Kein Treffer für 4 Bremen-Hamburg
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 36 Lüttich
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 38 Breslau
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 39 Brixen
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 47 Leitmeritz
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 49 Olmütz
└ @ Main In[35]:9
┌ Warning: großer Abstand 1.468088191811718 für 51 Straßburg
└ @ Main In[35]:15
┌ Warning: großer Abstand 4.122726152202933 für 52 Trient
└ @ Main In[35]:15
┌ Warning: Kein Treffer für 54 Wien
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 57 Pomesanien
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 59 Samland
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 60 Ermland
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 63 Reval
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 67 Ösel-Wiek
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 69 Kurland
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 71 Pedena
└ @ Main In[35]:9
┌ Warning: Kein Treffer für 72 Leitomis

In [93]:
names(df_map)

9-element Vector{String}:
 "id"
 "name"
 "bishopric_seat"
 "place_name"
 "longitude"
 "latitude"
 "num_longitude"
 "num_latitude"
 "new_place_id"

In [165]:
df_map_name = leftjoin(select(df_map, Not([:longitude, :latitude])), df_coo, on = :new_place_id => :id);

In [166]:
filter(:new_name => ismissing, df_map_name[!, [:id, :name, :place_name, :new_name]])

,id,name,place_name,new_name
,Int32,String?,String?,String?
1,4,Bremen-Hamburg,Bremen,missing
2,36,Lüttich,Lüttich (Liège),missing
3,38,Breslau,Breslau,missing
4,39,Brixen,Brixen,missing
5,47,Leitmeritz,Leitmeritz (Litoměřice),missing
6,49,Olmütz,Olmütz,missing
7,54,Wien,Wien,missing
8,57,Pomesanien,Marienwerder (Kwidzyn),missing
9,59,Samland,Königsberg (Kaliningrad),missing


Orte für die nicht zugeordneten Diözesen ergänzen.

In [167]:
findall((i) -> i in (4, 36, 38, 39, 47, 49, 54, 57, 59, 60, 63, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78), df_map[!, :id])

21-element Vector{Int64}:
  4
 36
 38
 39
 47
 49
 54
 56
 58
 59
 62
 66
 68
 70
 71
 72
 73
 74
 75
 76
 77

In [146]:
df_map[77, [:id, :name, :place_name, :new_place_id]] # df_map[x, :new_place_id] = ?

,id,name,place_name,new_place_id
,Int32,String?,String?,Int64
77,78,Königgrätz,Königgrätz,0


In [53]:
df_map[4, :new_place_id] = 402537 # Hamburg

402537

In [54]:
df_map[36, :new_place_id] = 443353 # Lüttich, Liège

443353

In [55]:
df_map[38, :new_place_id] = 659147 # Breslau, Wrocław

659147

In [56]:
df_map[39, :new_place_id] = 545680 # Brixen, Bressanone

545680

In [57]:
df_map[47, :new_place_id] = 693691 # Leitmeritz, Litoměřice

693691

In [58]:
df_map[49, :new_place_id] = 690063 # Olmütz, Olomouc

690063

In [59]:
df_map[54, :new_place_id] = 608906 # Wien, Vienna

608906

In [60]:
df_map[56, :new_place_id] = 671711 # Pomesanien, Kwidzyn

671711

In [61]:
df_map[58, :new_place_id] = 942885 # Königsberg, Kaliningrad

942885

In [62]:
df_map[59, :new_place_id] = 677866 # Ermland, Braniewo

677866

In [63]:
df_map[62, :new_place_id] = 734738 # Riga, Tallinn

734738

In [64]:
df_map[66, :new_place_id] = 737699 # Hapsal, Haapsalu

737699

In [65]:
df_map[68, :new_place_id] = 704058 # Kurland, Aizpute

704058

In [66]:
df_map[70, :new_place_id] = 726008 # Pedena, Pican

726008

In [67]:
df_map[71, :new_place_id] = 693692 # Leitomischl, Litomyšl

693692

In [68]:
df_map[72, :new_place_id] = 701994 # Semgallen, Mežotne

701994

In [69]:
df_map[73, :new_place_id] = 698174 # Budweis, České Budějovice

698174

In [70]:
df_map[74, :new_place_id] = 398266 # Corvey, Höxter

398266

In [71]:
df_map[75, :new_place_id] = 734797 # Dorpat, Tartu

734797

In [72]:
df_map[76, :new_place_id] = 541698 # Görz, Gorizia

541698

In [73]:
df_map[77, :new_place_id] = 696819 # Königgrätz, Hradec Králové

696819

Der Bischofssitz von Kulm ist nicht in einem der gleichnamigen Orte, sondern in Chełmża

In [74]:
findall(df_map[!, :id] .== 58)

1-element Vector{Int64}:
 57

In [75]:
df_map_name[df_map_name[!, :id] .== 58, [:id, :name, :place_name, :new_place_id, :new_name]]

,id,name,place_name,new_place_id,new_name
,Int32,String?,String?,Int64,String?
1,58,Kulm,Culmsee (Chełmża),682910,Kulm


Korrektur für den Bischofssitz von Kulm

In [76]:
df_map[57, :new_place_id] = 678667 # Kulm, Chełmża

678667

Korrektur für den Bischofssitz von Straßburg

In [38]:
findall(df_map[!, :id] .== 51)

1-element Vector{Int64}:
 51

In [40]:
df_map[51, [:id, :name, :place_name, :new_place_id]]

,id,name,place_name,new_place_id
,Int32,String?,String?,Int64
51,51,Straßburg,Straßburg,361556


In [41]:
df_map[51, :new_place_id] = 456258 # Straßburg, Strasbourg

456258

Korrektur für den Bischofssitz von Trient

In [43]:
findall(df_map[!, :id] .== 52)

1-element Vector{Int64}:
 52

In [44]:
df_map[52, [:id, :name, :place_name, :new_place_id]]

,id,name,place_name,new_place_id
,Int32,String?,String?,Int64
52,52,Trient,Trient,594968


In [45]:
df_map[52, :new_place_id] = 535120 # Trient, Trento

535120

Korrektur für den Bischofssitz von Brandenburg; schon in der Quelle falsch!?

In [46]:
findall(df_map[!, :name] .== "Brandenburg")

1-element Vector{Int64}:
 3

In [47]:
df_map[3, [:id, :name, :place_name, :new_place_id]]

,id,name,place_name,new_place_id
,Int32,String?,String?,Int64
3,3,Brandenburg,Brandenburg an der Havel,420202


In [48]:
df_map[3, :new_place_id] = 420204 # Brandenburg, Brandenburg an der Havel

420204

Korrektur für den Bischofssitz von Triest; schon in der Quelle falsch!?

In [91]:
findall(df_map[!, :name] .== "Triest")

1-element Vector{Int64}:
 78

In [93]:
df_map[78, :new_place_id] = 535158 # Triest, Trieste

535158

Ortsnamen prüfen

In [94]:
df_map_name = leftjoin(select(df_map, Not([:longitude, :latitude])), df_coo, on = :new_place_id => :id);

In [95]:
df_map_name[61:end, [:id, :name, :place_name, :new_place_id, :new_name]]

,id,name,place_name,new_place_id,new_name
,Int32,String?,String?,Int64,String?
1,43,Gurk,Gurk,617982,Gurk
2,38,Breslau,Breslau,659147,Wrocław
3,57,Pomesanien,Marienwerder (Kwidzyn),671711,Kwidzyn
4,60,Ermland,Braunsberg (Braniewo),677866,Braniewo
5,58,Kulm,Culmsee (Chełmża),678667,Chełmża
6,49,Olmütz,Olmütz,690063,Olomouc
7,47,Leitmeritz,Leitmeritz (Litoměřice),693691,Litoměřice
8,72,Leitomischl,Litomyšl,693692,Litomyšl
9,78,Königgrätz,Königgrätz,696819,Hradec Králové


Neue Orts-IDs in die Datenbank übernehmen.

In [96]:
DBInterface.execute(Wds.dbwiag, "DROP TABLE IF EXISTS dioc_place")
sql_tmp = "CREATE TEMPORARY TABLE dioc_place " *
"(id INT, " *
" new_place_id INT)"
DBInterface.execute(Wds.dbwiag, sql_tmp);

In [97]:
Wds.filltable!("dioc_place", df_map[!, [:id, :new_place_id]], clear_table=true)

┌ Info: Rows inserted: 78
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


78

In [98]:
sql_update = "UPDATE diocese as d, (SELECT id, new_place_id FROM dioc_place) as dp " *
"SET d.bishopric_seat_id = dp.new_place_id " *
"WHERE d.id = dp.id"
DBInterface.execute(Wds.dbwiag, sql_update);

Einträge prüfen

In [99]:
sql = "SELECT d.id, d.name, pl.id as id_place, pl.name as name_place, country_code " *
" FROM diocese AS d JOIN place AS pl on d.bishopric_seat_id = pl.id"
df_check = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [100]:
df_check[64:end, :]

,id,name,id_place,name_place,country_code
,Int32,String?,Int32,String?,String?
1,65,Wiener Neustadt,608914,Wiener Neustadt,AT
2,66,Seckau,610720,Seckau,AT
3,67,Ösel-Wiek,737699,Haapsalu,EE
4,68,Toul,455328,Toul,FR
5,69,Kurland,704058,Aizpute,LV
6,70,Aquileia,546681,Aquileia,IT
7,71,Pedena,726008,Pićan,HR
8,72,Leitomischl,693692,Litomyšl,CZ
9,73,Semgallen,701994,Mežotne,LV


## Literaturverweise

Quelle: Tabelle `wiag.diocese`

Die Referenz für Bistümer ist immer Nummer 1: Gatz, Erwin (Hg.)/Brodkorb, Clemens/Flachenecker, Helmut (Bearb.), Die Bistümer des Heiligen Römischen Reiches. Von ihren Anfängen bis zur Säkularisation, Freiburg i. Br. 2003

Die Quelle gibt es bei Bischöfen und Domherren auch. Kopiere die Quelle der Domherren.

In [11]:
table_name = "reference_volume";
item_type_dh = 5;
sql = "SELECT * FROM $(table_name) WHERE item_type_id = $(item_type_dh) AND reference_id = 2";
df_vol = Wds.sql_df(sql)

,id,comment,full_citation
,Int32,String?,String
1,14,missing,"Erwin Gatz (Hg.)/Clemens Brodkorb/Helmut Flachenecker (Bearb.), Die Bistümer des Heiligen Römischen Reiches. Von ihren Anfängen bis zur Säkularisation, Freiburg i. Br. 2003"


In [12]:
item_type_id = 1;
df_vol[1, :item_type_id] = item_type_id;

In [13]:
table_name = "reference_volume"
Wds.filltable!(table_name, select(df_vol, Not(:id)))

┌ Info: Rows inserted: 1
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


1

Alte Einträge über `item_type_id` löschen

In [12]:
item_type_id = 1;
sql = "DELETE FROM item_reference WHERE item_type_id = $(item_type_id)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM item_reference WHERE item_type_id = 1", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

IDs auslesen

In [15]:
sql = "SELECT id, id_in_source FROM item where item_type_id = $(item_type_id)"
df_idx = Wds.sql_df(sql);

Spalte für die Zusammenführung befüllen (Typ: String!)

In [16]:
df_dioc[!, :id_in_source] .= string.(df_dioc[!, :id_diocese]);

In [18]:
df_dioc_db = innerjoin(df_idx, df_dioc, on = :id_in_source);

In [20]:
size(df_dioc_db)

(79, 16)

Literaturverweise eintragen

In [19]:
columns = [
    :id => :item_id,
    :gatz_pages => :page,
]

2-element Vector{Pair{Symbol, Symbol}}:
         :id => :item_id
 :gatz_pages => :page

In [21]:
df_dioc_ref = select(df_dioc_db, columns);

In [23]:
df_dioc_ref[!, :reference_id] .= 2;

In [25]:
df_dioc_ref[!, :item_type_id] .= item_type_id;

In [27]:
table_name = "item_reference";
Wds.filltable!(table_name, df_dioc_ref, clear_table = false)

┌ Info: Rows inserted: 79
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1204


79